In [160]:
import pandas as pd
import numpy as np
from functools import reduce
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from matplotlib import pyplot as plt
from tqdm import tqdm
from pathlib import Path
import datetime
import dateutil
from typing import List
from pandas.tseries.frequencies import to_offset

In [177]:
path = Path('..\samples\Svodniy_otchet_po_chasam_intensivnost+skorost_01_01_2024_31_12_202411111111.xlsx')
initial_df = pd.read_excel(path, header=[2,3])

In [178]:
df = initial_df.copy()

In [179]:
df = df.set_index(df.columns[0])
df.index = pd.to_datetime(df.index, 'coerce', dayfirst=True)
df = df[~df.index.isna()]

C:\Users\riabt\AppData\Local\Temp\ipykernel_22352\3710854130.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df.index = pd.to_datetime(df.index, 'coerce', dayfirst=True)


In [180]:
df.head()

Общая интенсивность автомобилей                  \
                                                     Итого Прямое Обратное   
(Дата, Unnamed: 0_level_1)                                                   
2024-01-01 00:59:59                                  169.0  112.0     57.0   
2024-01-01 01:59:59                                  261.0  156.0    105.0   
2024-01-01 02:59:59                                  251.0  147.0    104.0   
2024-01-01 03:59:59                                  168.0   94.0      NaN   
2024-01-01 04:59:59                                  138.0   74.0      NaN   

                           Легковые (до 6 м)                  \
                                       Итого Прямое Обратное   
(Дата, Unnamed: 0_level_1)                                     
2024-01-01 00:59:59                    167.0  110.0     57.0   
2024-01-01 01:59:59                    260.0  156.0    104.0   
2024-01-01 02:59:59                    251.0  147.0    104.0   
2024-01-01 03:59:59                    167.0   94.0     73.0   
2024-01-01 04:59:59                    136.0   72.0     64.0   

                           Малые груз. (6-9 м)                  \
                                         Итого Прямое Обратное   
(Дата, Unnamed: 0_level_1)                                       
2024-01-01 00:59:59                        1.0    1.0      0.0   
2024-01-01 01:59:59                        1.0    0.0      1.0   
2024-01-01 02:59:59                        0.0    0.0      0.0   
2024-01-01 03:59:59                        1.0    0.0      1.0   
2024-01-01 04:59:59                        1.0    1.0      0.0   

                           Грузовые (9-13 м)  ... Автобусы                  \
                                       Итого  ...    Итого Прямое Обратное   
(Дата, Unnamed: 0_level_1)                    ...                            
2024-01-01 00:59:59                      1.0  ...      0.0    0.0      0.0   
2024-01-01 01:59:59                      0.0  ...      0.0    0.0      0.0   
2024-01-01 02:59:59                      0.0  ...      0.0    0.0      0.0   
2024-01-01 03:59:59                      0.0  ...      0.0    0.0      0.0   
2024-01-01 04:59:59                      1.0  ...      0.0    0.0      0.0   

                           Мотоциклы                 Скорость, км/ч           \
                               Итого Прямое Обратное         Прямое Обратное   
(Дата, Unnamed: 0_level_1)                                                     
2024-01-01 00:59:59              0.0    0.0      0.0          107.0    105.0   
2024-01-01 01:59:59              0.0    0.0      0.0          101.0    112.0   
2024-01-01 02:59:59              0.0    0.0      0.0          105.0    107.0   
2024-01-01 03:59:59              0.0    0.0      0.0           94.0    109.0   
2024-01-01 04:59:59              0.0    0.0      0.0           93.0     96.0   

                           Загрузка, %           
                                Прямое Обратное  
(Дата, Unnamed: 0_level_1)                       
2024-01-01 00:59:59                1.0      0.0  
2024-01-01 01:59:59                1.0      1.0  
2024-01-01 02:59:59                2.0      1.0  
2024-01-01 03:59:59                1.0      0.0  
2024-01-01 04:59:59                1.0      0.0  

[5 rows x 28 columns]

In [150]:
def load_data(initial_df):
    df = initial_df.copy()
    title = df.iloc[3]['Unnamed: 0']
    df = df.drop([2, 3])
    data_groups = {}
    current_group = df.iloc[0].values[0]

    for index, value in zip(df.iloc[0].index, df.iloc[0].values):
        if (pd.isna(value)):
            data_groups[current_group].append(index)
        else:
            current_group = value
            data_groups[current_group] = [index]

    data_dfs = []
    index = pd.to_datetime(df['Unnamed: 0'][2:-3], dayfirst=True)

    for group, cols in tqdm(zip(list(data_groups.keys())[1:], list(data_groups.values())[1:])):
        data_df = df[cols].copy()
        data_df.columns = data_df.iloc[1].values
        data_df = data_df.drop([0, 1])[:-3]
        data_df = data_df.set_index(index)
        data_df.index = pd.to_datetime(data_df.index)
        data_df.index.names = ['timestamp']
        data_dfs.append((group, data_df))
    
    return title, data_groups, data_dfs

In [151]:
df[df.columns[1]]['2024-07-16 06:59:59']

np.float64(408.0)

In [152]:
def get_nan_intervals(series: pd.Series) -> List[pd.Series]:
    na_mask = series.isna()
    groups = (na_mask != na_mask.shift()).cumsum()
    nan_intervals = series[na_mask].groupby(groups[na_mask])
    # def expand_interval(interval: pd.Series) -> pd.Series:
    #     left_i = interval.index.get_loc(interval.index[0])
    #     right_i = interval.index.get_loc(interval.index[-1])
    #     return series.iloc[]

    # return list(map(expand_interval, nan_intervals))
    return [interval for _, interval in nan_intervals]

In [157]:
def predict2(data: pd.DataFrame) -> pd.Series:
    df_empty = data.copy()
    df_empty.loc[:, :] = np.nan
    for col_name in data.columns:
        col = data[col_name]
        nan_intervals = get_nan_intervals(col)
        if (len(nan_intervals) == 0):
            continue
        for interval in nan_intervals:
            if (len(interval)) == 0:
                continue
            # конец заполненных данных
            avaiable_train_end_index = col.index.get_indexer([interval.index[0]])
            # доступные для обучения
            avaiable_train = col.iloc[:avaiable_train_end_index[0]]
            delta = pd.Timedelta(days=avaiable_train.index[-1].weekday())
            # timestamp начала недели
            week_start = (avaiable_train.index[-1] - delta).normalize()
            # индекс начала недели
            train_start = avaiable_train.index.get_indexer([week_start], method='backfill')
            # трейн данные с начала недели
            train = avaiable_train[train_start[0]:]
            seasonal_periods = 24
            min_len = seasonal_periods * 2
            if len(train) < min_len:
                # добавляются данные с прошлой недели если не хватате для обучения
                if (len(avaiable_train[:train.index[0]][:-1]) < min_len - len(train)):
                    continue
                else:
                    train = pd.concat([avaiable_train[:train.index[0]][:-1][:len(train) - (min_len+1):-1],train]).sort_index()
            seasonal_periods = 24
            eps = 10e-03
            train = np.log(train.fillna(train.mean()).map(lambda x: max(x, 1)) + eps)
            fit = ExponentialSmoothing(
                train.values,
                trend=None,
                seasonal='mul',
                seasonal_periods=seasonal_periods,
                damped_trend=False,
            ).fit()
            left_i = col.index.get_loc(interval.index[0])
            right_i = col.index.get_loc(interval.index[-1])
            # предикт данных включая граничные значения
            predict_index = col.iloc[left_i-1:right_i+2].index
            # запись
            df_empty.loc[predict_index,col_name] = np.exp(fit.forecast(len(interval) + 2))
            # df.loc[predict_index,col_name] = np.exp(int(fit.forecast(len(interval) + 2)))

    return df_empty


# predicted = predict2(df)
# test_col = df[df.columns[2]]
# test_interval = get_nan_intervals(test_col)[1]
# test_interval
predicted = predict2(df)

d:\projects\PycharmProjects\transportation\venv\lib\site-packages\statsmodels\tsa\holtwinters\model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
d:\projects\PycharmProjects\transportation\venv\lib\site-packages\statsmodels\tsa\holtwinters\model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(


In [183]:
initial_df = initial_df.set_index(initial_df.columns[0])
initial_df.index = pd.to_datetime(initial_df.index, 'coerce', dayfirst=True)
initial_df = initial_df[~initial_df.index.isna()]
initial_df

C:\Users\riabt\AppData\Local\Temp\ipykernel_22352\4012838070.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  initial_df.index = pd.to_datetime(initial_df.index, 'coerce', dayfirst=True)


Общая интенсивность автомобилей                  \
                                                     Итого Прямое Обратное   
(Дата, Unnamed: 0_level_1)                                                   
2024-01-01 00:59:59                                  169.0  112.0     57.0   
2024-01-01 01:59:59                                  261.0  156.0    105.0   
2024-01-01 02:59:59                                  251.0  147.0    104.0   
2024-01-01 03:59:59                                  168.0   94.0      NaN   
2024-01-01 04:59:59                                  138.0   74.0      NaN   
...                                                    ...    ...      ...   
2024-12-31 19:59:59                                  617.0  351.0    266.0   
2024-12-31 20:59:59                                  481.0  261.0    220.0   
2024-12-31 21:59:59                                  369.0  189.0    180.0   
2024-12-31 22:59:59                                  316.0  180.0    136.0   
2024-12-31 23:59:59                                  214.0  102.0    112.0   

                           Легковые (до 6 м)                  \
                                       Итого Прямое Обратное   
(Дата, Unnamed: 0_level_1)                                     
2024-01-01 00:59:59                    167.0  110.0     57.0   
2024-01-01 01:59:59                    260.0  156.0    104.0   
2024-01-01 02:59:59                    251.0  147.0    104.0   
2024-01-01 03:59:59                    167.0   94.0     73.0   
2024-01-01 04:59:59                    136.0   72.0     64.0   
...                                      ...    ...      ...   
2024-12-31 19:59:59                    596.0  333.0    263.0   
2024-12-31 20:59:59                    475.0  257.0    218.0   
2024-12-31 21:59:59                    365.0  187.0    178.0   
2024-12-31 22:59:59                    312.0  178.0    134.0   
2024-12-31 23:59:59                    210.0   99.0    111.0   

                           Малые груз. (6-9 м)                  \
                                         Итого Прямое Обратное   
(Дата, Unnamed: 0_level_1)                                       
2024-01-01 00:59:59                        1.0    1.0      0.0   
2024-01-01 01:59:59                        1.0    0.0      1.0   
2024-01-01 02:59:59                        0.0    0.0      0.0   
2024-01-01 03:59:59                        1.0    0.0      1.0   
2024-01-01 04:59:59                        1.0    1.0      0.0   
...                                        ...    ...      ...   
2024-12-31 19:59:59                        5.0    4.0      1.0   
2024-12-31 20:59:59                        3.0    1.0      2.0   
2024-12-31 21:59:59                        1.0    1.0      0.0   
2024-12-31 22:59:59                        2.0    1.0      1.0   
2024-12-31 23:59:59                        3.0    2.0      1.0   

                           Грузовые (9-13 м)  ... Автобусы                  \
                                       Итого  ...    Итого Прямое Обратное   
(Дата, Unnamed: 0_level_1)                    ...                            
2024-01-01 00:59:59                      1.0  ...      0.0    0.0      0.0   
2024-01-01 01:59:59                      0.0  ...      0.0    0.0      0.0   
2024-01-01 02:59:59                      0.0  ...      0.0    0.0      0.0   
2024-01-01 03:59:59                      0.0  ...      0.0    0.0      0.0   
2024-01-01 04:59:59                      1.0  ...      0.0    0.0      0.0   
...                                      ...  ...      ...    ...      ...   
2024-12-31 19:59:59                      5.0  ...      0.0    0.0      0.0   
2024-12-31 20:59:59                      2.0  ...      0.0    0.0      0.0   
2024-12-31 21:59:59                      1.0  ...      0.0    0.0      0.0   
2024-12-31 22:59:59                      1.0  ...      0.0    0.0      0.0   
2024-12-31 23:59:59                      1.0  ...      0.0    0.0      0.0   

                   

In [188]:
with pd.ExcelWriter( f'{Path.joinpath(path.parent,"Обработано__" + path.stem + ".xlsx")}', engine='xlsxwriter', datetime_format="yyyy-mm-dd HH:MM:SS") as writer:
    initial_df.to_excel(excel_writer=writer, sheet_name='Исходные данные', merge_cells=True)
    predicted = predicted.apply(pd.to_numeric, errors='coerce').round().astype('Int64')
    predicted.to_excel(excel_writer=writer, sheet_name='Результаты', merge_cells=True)
    predicted.combine_first(initial_df).to_excel(excel_writer=writer, sheet_name='Объединенные данные', merge_cells=True)
    
    date_format = writer.book.add_format({'num_format': 'yyyy-mm-dd hh:mm:ss'})
    for sheet in writer.sheets:
        writer.sheets[sheet].set_column('A:A', 20, date_format)  
    # # initial data
    # initial_df.to_excel(excel_writer=writer,
    #                      sheet_name='Исходные данные')
    # # resulted
    # numeric_cols = resulted_df.select_dtypes(include='float').columns
    # resulted_df[numeric_cols] = resulted_df[numeric_cols].round(0)
    # resulted_df = resulted_df.reset_index().rename(columns={'timestamp': 'Дата'})
    # resulted_df['Дата'] = resulted_df['Дата'].astype(str)
    # resulted_df.to_excel(excel_writer=writer,
    #                      sheet_name='Результаты')
    # # merged
    # d_initial = initial_df[4:-3].copy().reset_index(drop=True)
    # d_initial.columns = resulted_df.columns
    # filled = resulted_df.combine_first(d_initial)
    # filled.to_excel(excel_writer=writer,
    #                      sheet_name='Объединенные данные')

In [193]:
def predict(df, title):
    predicted_df = pd.DataFrame(index=df.index)
    for col in df.columns:
        data = df[col].to_frame().sort_index()
        data['is_nan'] = data[col].isna().astype(int)
        data['group'] = (data['is_nan'].diff() == 1).cumsum()
        data = data.reset_index()
        nan_intervals = data[data['is_nan'] == 1].groupby(
            'group').agg(start=('timestamp', 'first'), end=('timestamp', 'last'))
        data = data.set_index('timestamp')
        data = data.drop(['is_nan', 'group'], axis=1)
        data = data.astype(float)
        nan_intervals['duration'] = nan_intervals['end'] - nan_intervals['start']
        if (len(nan_intervals) == 0):
            predicted_df[f'{title} - {col} - predict'] = pd.Series(
                    index=predicted_df.index)
            continue
        for interval in nan_intervals.itertuples():
            if (len(interval) == 0):
                predicted_df[f'{col} - predict'] = pd.Series(
                    index=predicted_df.index)
                continue
            start, end, duration = interval.start, interval.end, interval.duration
            
            if (duration < pd.Timedelta(hours=5)):
                i_before = data.index.get_indexer([start]) - 1
                i_after = data.index.get_indexer([end]) + 1
                data.iloc[i_before] = np.nan
                data.iloc[i_after] = np.nan
                interpolation = data[start - pd.Timedelta(hours=12):end + pd.Timedelta(hours=12)
                     ].interpolate(method='spline', order=2)
                prediction_index =  data.index[i_before[0]:i_after[0] + 1]
                predicted_df.loc[prediction_index,
                                 f'{title} - {col}: predict'] = interpolation[prediction_index]
            
            train_end = data[:start].index[-2]
            train_start = (
                train_end - pd.Timedelta(days=train_end.weekday())).normalize()
            train_start = data.iloc[data.index.get_indexer(
                [train_start], method='nearest')[0]].name
            train = data[train_start:train_end]
            seasonal_periods = 24
            eps = 10e-05
            if len(train) < seasonal_periods * 2:
                train = pd.concat( 
                    [train, data[:train_start][len(train) - (seasonal_periods*2+1):-1]]).sort_index()
            train = np.log(train.replace(0,1) + eps)
            fit = ExponentialSmoothing(
                train.values,
                trend=None,
                seasonal='mul',
                seasonal_periods=seasonal_periods,
                damped_trend=False,
            ).fit()
            predict_index = data.index[data.index.get_loc(
                start):data.index.get_loc(end) + 2]
            nan_counts = len(data[start:end])
            data[train_start:train_end] = data[train_start:train_end] + eps
            predicted_df.loc[predict_index,
                                 f'{title} - {col}: predict'] = np.exp(fit.forecast(int(nan_counts + 1)))
                
    return predicted_df


for col in df.columns:
    df[col] = predict(df[col])

# resulted_df = pd.DataFrame(index=data_dfs[0][1].index)
# for name, df in data_dfs:
#     predicted = predict(df, name)
#     resulted_df = pd.concat([resulted_df, predicted], axis=1)

In [194]:
data_dfs[0][1]['Обратное'] + 10e-05

timestamp
2024-01-01 00:59:59     65.0001
2024-01-01 01:59:59    134.0001
2024-01-01 02:59:59    106.0001
2024-01-01 03:59:59     73.0001
2024-01-01 04:59:59     42.0001
                         ...   
2024-12-31 19:59:59    284.0001
2024-12-31 20:59:59    212.0001
2024-12-31 21:59:59    186.0001
2024-12-31 22:59:59    130.0001
2024-12-31 23:59:59     47.0001
Name: Обратное, Length: 8771, dtype: object

In [195]:
for name, df in data_dfs:
    print(name)

Общая интенсивность автомобилей
Легковые (до 6 м)
Малые груз. (6-9 м)
Грузовые (9-13 м)
Груз. большие (13-22 м)
Автопоезда (22-30 м)
Автобусы
Мотоциклы
Скорость, км/ч
Загрузка, %


In [196]:
# = pd.DataFrame(index=data_dfs[0][1].index)
# resulted_df['Общая интенсивность автомобилей - Итого: predict'][~resulted_df['Общая интенсивность автомобилей - Итого: predict'].isna()]
# for name, df in data_dfs:
#     resulted_df = pd.concat([resulted_df, df], axis=1)
    

In [197]:
info_row = [np.nan]
for col_name, cols in zip(list(data_groups.keys())[1:], list(data_groups.values())[1:]):
    info_row += [col_name] * len(cols)

In [198]:
list(zip(info_row, resulted_df.columns))

[(nan, 'Общая интенсивность автомобилей - Итого - predict'),
 ('Общая интенсивность автомобилей',
  'Общая интенсивность автомобилей - Прямое - predict'),
 ('Общая интенсивность автомобилей',
  'Общая интенсивность автомобилей - Обратное: predict'),
 ('Общая интенсивность автомобилей', 'Легковые (до 6 м) - Итого - predict'),
 ('Легковые (до 6 м)', 'Легковые (до 6 м) - Прямое - predict'),
 ('Легковые (до 6 м)', 'Легковые (до 6 м) - Обратное: predict'),
 ('Легковые (до 6 м)', 'Малые груз. (6-9 м) - Итого - predict'),
 ('Малые груз. (6-9 м)', 'Малые груз. (6-9 м) - Прямое - predict'),
 ('Малые груз. (6-9 м)', 'Малые груз. (6-9 м) - Обратное: predict'),
 ('Малые груз. (6-9 м)', 'Грузовые (9-13 м) - Итого - predict'),
 ('Грузовые (9-13 м)', 'Грузовые (9-13 м) - Прямое - predict'),
 ('Грузовые (9-13 м)', 'Грузовые (9-13 м) - Обратное: predict'),
 ('Грузовые (9-13 м)', 'Груз. большие (13-22 м) - Итого - predict'),
 ('Груз. большие (13-22 м)', 'Груз. большие (13-22 м) - Прямое - predict'),
 ('

In [199]:
len(info_row), resulted_df.shape

(29, (8771, 28))

In [200]:
save_path = str(f'{pathlib.Path(path).parent}/output/{title.replace("/", "-").replace(" ", "_")}__{datetime.datetime.now().strftime("%d-%m-%Y__%H-%M-%S")}.xslx')
save_path

'..\\samples/output/км_32+000_а-д_А-146_Краснодар-Верхнебаканский___31-03-2025__04-20-27.xslx'

In [202]:
with pd.ExcelWriter(f'..{path.split(".")[-2]}-2.xlsx', engine='xlsxwriter') as writer:
    # initial data
    initial_df.to_excel(excel_writer=writer,
                         sheet_name='Исходные данные')
    # resulted
    numeric_cols = resulted_df.select_dtypes(include='float').columns
    resulted_df[numeric_cols] = resulted_df[numeric_cols].round(0)
    resulted_df = resulted_df.reset_index().rename(columns={'timestamp': 'Дата'})
    resulted_df['Дата'] = resulted_df['Дата'].astype(str)
    resulted_df.to_excel(excel_writer=writer,
                         sheet_name='Результаты')
    # merged
    d_initial = initial_df[4:-3].copy().reset_index(drop=True)
    d_initial.columns = resulted_df.columns
    filled = resulted_df.combine_first(d_initial)
    filled.to_excel(excel_writer=writer,
                         sheet_name='Объединенные данные')

In [210]:
from pathlib import Path

Path('../samples/report1.xlsx').suffix

'.xlsx'